In [ ]:
import pandas as pd
from pandas import DataFrame
import requests
from bs4 import BeautifulSoup
import datetime as dt

In [ ]:
# 수집할 음원 사이트의 주소 페이지 설정
# 1~50, 51~100으로 2개의 페이지로 나눠져 있음
pages = ['1', '2']

# dict 형식의 노래제목, 가수, 앨범, 순위를 담을 빈 list 생성
genie_chart = []

for p in pages:
    
    # 수집할 컨텐츠가 있는 날짜별 웹 페이지 주소
    date = dt.datetime.now().strftime("%Y%m%d")
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh=09&rtm=N&pg={p}'
    # print(url)

    # 브라우저 버전정보
    userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    # 접속 객체 생성
    # request는 직접 header 정보 갱신이 불가능해 session 생성
    session = requests.Session()

    # 접속 객체에 부가정보(header) 삽입, 크롤링이 가능하도록 유도
    session.headers.update({
        "Referer": "",
        "User-Agent": userAgent
    })

    # 생성한 접속 객체를 활용해 웹 페이지에 접속
    # 지정된 URL에 GET 요청을 보내고
    # 그에 대한 응답을 r 변수에 할당
    r = session.get(url)

    # 접속 실패 시 에러 코드와 에러 메시지 출력
    if r.status_code != 200:
        msg = "[%d Error] %s 에러 발생" % (r.status_code, r.reason)
        raise Exception(msg)

    # 인코딩 형식을 지정해 beautifulsoup 객체를 생성
    r.encoding = 'utf-8'

    # r 변수에 저장된 text 정보를 시각화, 확인
    # print(r.text)

    # r.text에서 HTML을 파싱하고,
    # 파싱된 HTML을 나타내는
    # BeautifulSoup 객체 soup를 생성
    soup = BeautifulSoup(r.text)
    print(soup)